In [1]:
import networkx as nx

from regraph import Rule, plot_rule
from regraph.neo4j.graphs import Neo4jGraph
from regraph.neo4j.cypher_utils import *

In [2]:
# initialize the neo4j driver, wrappped into Neo4jGraph object
g = Neo4jGraph(uri="bolt://localhost:7687", user="neo4j", password="admin")
g._clear()

In [3]:
g.add_nodes_from([
    ("n1", {"name": "Nils"}),
    ("n2", {"acmid": 220})
])
g.add_edge("n1", "n2", {"contact": True})


In [4]:
pattern = nx.DiGraph()
pattern.add_nodes_from(["a", "b", "c", "d"])
pattern.add_edges_from([("a", "c"), ("a", "d"), ("d", "c")])

In [5]:
rule = Rule.from_transform(pattern)

In [6]:
rule.inject_remove_edge("d", "c")
merge = rule.inject_merge_nodes(["c", "d"])
rule.inject_add_edge("b", merge)

In [7]:
instance = {
    "a": "n6",
    "b": "n10",
    "c": "n5",
    "d": "n9"
}

In [8]:
rule.to_cypher(instance)

// Match nodes and edges of the instance 
MATCH (lhs_a:node { id : 'n6'}) , (lhs_b:node { id : 'n10'}) , (lhs_c:node { id : 'n5'}) , (lhs_d:node { id : 'n9'})  , (lhs_a)-[lhs_a_lhs_c:edge]->(lhs_c), (lhs_a)-[lhs_a_lhs_d:edge]->(lhs_d), (lhs_d)-[lhs_d_lhs_c:edge]->(lhs_c)

// Removing edge 'd->c' of the lhs 
DELETE lhs_d_lhs_c

WITH lhs_a, lhs_d, lhs_a_lhs_c, lhs_c, lhs_a_lhs_d, lhs_b // Renaming vars to correspond to the vars of P
WITH lhs_a_lhs_c, lhs_a_lhs_d , lhs_a as p_a, lhs_b as p_b, lhs_c as p_c, lhs_d as p_d 

// Merging nodes '{'c', 'd'}' of the preserved part into 'c_d' 
// accumulate all the attrs of the nodes to be merged
WITH [] as new_props, lhs_a_lhs_c, lhs_a_lhs_d, p_b, p_d, p_a, p_c
WITH new_props + REDUCE(pairs = [], k in keys(p_c) | 
	pairs + REDUCE(inner_pairs = [], v in p_c[k] | 
		inner_pairs + {key: k, value: v})) as new_props, lhs_a_lhs_c, lhs_a_lhs_d, p_b, p_d, p_a, p_c
WITH new_props + REDUCE(pairs = [], k in keys(p_d) | 
	pairs + REDUCE(inner_pairs = [], v in

("// Match nodes and edges of the instance \nMATCH (lhs_a:node { id : 'n6'}) , (lhs_b:node { id : 'n10'}) , (lhs_c:node { id : 'n5'}) , (lhs_d:node { id : 'n9'})  , (lhs_a)-[lhs_a_lhs_c:edge]->(lhs_c), (lhs_a)-[lhs_a_lhs_d:edge]->(lhs_d), (lhs_d)-[lhs_d_lhs_c:edge]->(lhs_c)\n\n// Removing edge 'd->c' of the lhs \nDELETE lhs_d_lhs_c\n\nWITH lhs_a, lhs_d, lhs_a_lhs_c, lhs_c, lhs_a_lhs_d, lhs_b // Renaming vars to correspond to the vars of P\nWITH lhs_a_lhs_c, lhs_a_lhs_d , lhs_a as p_a, lhs_b as p_b, lhs_c as p_c, lhs_d as p_d \n\n// Merging nodes '{'c', 'd'}' of the preserved part into 'c_d' \n// accumulate all the attrs of the nodes to be merged\nWITH [] as new_props, lhs_a_lhs_c, lhs_a_lhs_d, p_b, p_d, p_a, p_c\nWITH new_props + REDUCE(pairs = [], k in keys(p_c) | \n\tpairs + REDUCE(inner_pairs = [], v in p_c[k] | \n\t\tinner_pairs + {key: k, value: v})) as new_props, lhs_a_lhs_c, lhs_a_lhs_d, p_b, p_d, p_a, p_c\nWITH new_props + REDUCE(pairs = [], k in keys(p_d) | \n\tpairs + REDUCE(